In [ ]:
import math

def longest_palindromic_subsequence_length_table(s: str) -> list[list[int]]:
    """
    Computes the lengths of the Longest Palindromic Subsequence (LPS)
    for all possible substrings of the given string s.

    Args:
        s: The input string.

    Returns:
        A 2D list (DP table) where dp[i][j] stores the length of the
        LPS of the substring s[i...j].
    """
    n = len(s)
    # dp[i][j] stores the length of the LPS of substring s[i...j]
    # Initialize with 0s
    dp = [[0 for _ in range(n)] for _ in range(n)]

    # All single characters are palindromes of length 1
    for i in range(n):
        dp[i][i] = 1

    # Fill the DP table for increasing substring lengths (current_length)
    # current_length ranges from 2 to n
    for current_length in range(2, n + 1):
        # i is the starting index of the substring
        # Iterate from 0 up to n - current_length (inclusive)
        for i in range(n - current_length + 1):
            # j is the ending index of the substring
            j = i + current_length - 1

            # If the characters at the ends of the current substring match
            if s[i] == s[j]:
                # If the current_length is 2 (e.g., "aa", "bb"), the LPS is 2
                if current_length == 2:
                    dp[i][j] = 2
                else:
                    # If current_length > 2, the LPS includes s[i] and s[j]
                    # plus the LPS of the inner substring s[i+1...j-1]
                    dp[i][j] = 2 + dp[i + 1][j - 1]
            else:
                # If the characters at the ends do not match,
                # the LPS is the maximum of:
                # 1. LPS of substring s[i+1...j] (excluding s[i])
                # 2. LPS of substring s[i...j-1] (excluding s[j])
                dp[i][j] = max(dp[i + 1][j], dp[i][j - 1])
    return dp

def max_palindromic_subsequence_product_score(s: str) -> int:
    """
    Calculates the maximum possible score by building exactly two
    non-overlapping palindromic subsequences from the given word.
    The score is the product of the lengths of these two subsequences.

    Args:
        s: The input word composed of lowercase English letters.

    Returns:
        The maximum score achievable. Returns 0 if the string length
        is less than 2, as two non-overlapping subsequences cannot be formed.
    """
    n = len(s)

    # If the string length is less than 2, it's impossible to form
    # two distinct non-overlapping subsequences, each with at least one character.
    if n < 2:
        return 0

    # Precompute LPS lengths for all substrings using the helper function
    # lps_lengths_table[i][j] will store LPS length of s[i...j]
    lps_lengths_table = longest_palindromic_subsequence_length_table(s)

    max_score = 0

    # Iterate through all possible split points 'k'
    # 'k' is the last index of the first subsequence's substring (s[0...k])
    # The second subsequence's substring will then be s[k+1...n-1]
    # 'k' must be at least 0 and at most n-2 to ensure both parts have at least one character.
    for k in range(n - 1):
        # Get the LPS length for the first part (from index 0 to k)
        len1 = lps_lengths_table[0][k]

        # Get the LPS length for the second part (from index k+1 to n-1)
        len2 = lps_lengths_table[k + 1][n - 1]

        # Calculate the product score for the current split
        current_score = len1 * len2

        # Update the overall maximum score found so far
        max_score = max(max_score, current_score)

    return max_score

# Read input from stdin
word = input()

# Calculate and print the maximum score
result = max_palindromic_subsequence_product_score(word)
print(result)



ArgumentError: ArgumentError: Package math not found in current path.
- Run `import Pkg; Pkg.add("math")` to install the math package.

In [ ]:
import sys
from collections import deque

def solve():
    # Read N (number of nodes) and M (number of initial edges)
    N, M = map(int, sys.stdin.readline().split())

    # Adjacency list to represent the graph
    # Nodes are 1-indexed, so we use N+1 size
    adj = [[] for _ in range(N + 1)]
    for _ in range(M):
        u, v = map(int, sys.stdin.readline().split())
        adj[u].append(v)
        adj[v].append(u)

    # colors array:
    # -1: Uncolored
    # 0: Color A (e.g., White)
    # 1: Color B (e.g., Black)
    colors = [-1] * (N + 1)
    
    # visited array to keep track of nodes processed during BFS
    visited = [False] * (N + 1)
    
    # components_info stores (count_color_A, count_color_B) for each bipartite component
    components_info = []
    
    # component_representative_nodes stores one node from each identified component.
    # This is used later to add edges to connect components.
    component_representative_nodes = [] 

    # Iterate through all nodes to find and process all connected components
    for i in range(1, N + 1):
        if not visited[i]:
            # Start a Breadth-First Search (BFS) for a new component
            q = deque()
            q.append((i, 0)) # Add the starting node with an initial color (0)
            
            visited[i] = True
            colors[i] = 0 # Assign the initial color
            
            current_component_count_A = 1 # Count nodes with Color A in current component
            current_component_count_B = 0 # Count nodes with Color B in current component
            is_bipartite = True # Flag to check if the component is bipartite

            # Store this node as a representative for the current component
            component_representative_nodes.append(i) 

            while q:
                u, u_color = q.popleft() # Get current node and its color

                # Iterate through all neighbors of the current node
                for v in adj[u]:
                    if colors[v] == -1: # If neighbor 'v' is uncolored
                        colors[v] = 1 - u_color # Assign the opposite color
                        visited[v] = True # Mark as visited
                        
                        # Update counts for the current component
                        if colors[v] == 0:
                            current_component_count_A += 1
                        else:
                            current_component_count_B += 1
                        
                        q.append((v, colors[v])) # Add neighbor to queue for further processing
                    elif colors[v] == u_color: 
                        # If neighbor 'v' is already colored with the same color as 'u',
                        # then the graph is not bipartite.
                        is_bipartite = False
                        # The problem statement guarantees a valid reconstruction,
                        # implying that all components will be bipartite.
                        # So, this 'else if' branch should ideally not be hit for valid inputs.
            
            # After BFS completes for a component, if it's bipartite, store its color counts
            if is_bipartite:
                components_info.append((current_component_count_A, current_component_count_B))
            # If not bipartite, based on problem guarantees, we assume this case doesn't lead to an invalid state.
            # In a general bipartite problem, this would indicate no solution.

    # Dynamic Programming to find the minimum absolute difference |W - B|
    # dp[diff_val + offset] will be True if 'diff_val' (W-B) is achievable
    # The maximum possible difference is N (all nodes are one color)
    # The minimum possible difference is -N (all nodes are the other color)
    # So, the array size needs to accommodate differences from -N to N, which is 2*N + 1.
    # We use 'offset' to map negative differences to non-negative array indices.
    
    offset = N # Offset to shift differences to positive indices
    dp = [False] * (2 * N + 1)
    dp[offset] = True # Initial state: 0 nodes, 0 difference (before considering any components)

    # Iterate through each component's color counts
    for c0, c1 in components_info:
        # Create a new DP array for the current iteration to avoid using
        # values from the current component in its own calculation.
        new_dp = [False] * (2 * N + 1)
        
        # Option 1 for current component: c0 nodes are white, c1 nodes are black
        diff_option1 = c0 - c1
        # Option 2 for current component: c1 nodes are white, c0 nodes are black
        diff_option2 = c1 - c0

        # Iterate through all previously achievable differences
        for prev_diff_idx in range(2 * N + 1):
            if dp[prev_diff_idx]: # If this previous difference was achievable
                prev_diff = prev_diff_idx - offset # Convert index back to actual difference

                # Calculate new difference if Option 1 is chosen for current component
                current_diff1 = prev_diff + diff_option1
                # Check if the new difference index is within bounds
                if 0 <= current_diff1 + offset <= 2 * N:
                    new_dp[current_diff1 + offset] = True # Mark as achievable
                
                # Calculate new difference if Option 2 is chosen for current component
                current_diff2 = prev_diff + diff_option2
                # Check if the new difference index is within bounds
                if 0 <= current_diff2 + offset <= 2 * N:
                    new_dp[current_diff2 + offset] = True # Mark as achievable
        dp = new_dp # Update the main DP array for the next iteration

    # Find the minimum absolute difference from the final DP array
    min_abs_diff = float('inf') # Initialize with a very large value
    for diff_idx in range(2 * N + 1):
        if dp[diff_idx]: # If this difference is achievable
            # Calculate the absolute difference
            min_abs_diff = min(min_abs_diff, abs(diff_idx - offset))

    # Calculate the number of edges to add to connect all components
    # If there are 'C' components, we need to add 'C - 1' edges to connect them.
    num_components = len(components_info)
    added_edges_count = num_components - 1
    
    # Print the first line of output: minimum |W - B| and number of added edges
    sys.stdout.write(f"{min_abs_diff} {added_edges_count}\n")

    # Print the newly added edges, if any
    if added_edges_count > 0:
        # To connect all components, we can simply connect each component
        # (from the second one onwards) to a representative node of the first component.
        # This forms a star-like graph, ensuring overall connectivity.
        first_comp_node = component_representative_nodes[0]
        for i in range(1, num_components):
            other_comp_node = component_representative_nodes[i]
            sys.stdout.write(f"{first_comp_node} {other_comp_node}\n")

# Call the solve function to execute the program
solve()



ArgumentError: ArgumentError: Package sys not found in current path.
- Run `import Pkg; Pkg.add("sys")` to install the sys package.

In [1]:
from collections import defaultdict, deque
import sys

def is_bipartite_and_count(graph, n):
    # Colors: 1 (white), -1 (black), 0 (uncolored)
    colors = [0] * (n + 1)
    components = []
    
    for start in range(1, n + 1):
        if colors[start] != 0:
            continue
        # BFS to find a component and check bipartiteness
        white_count = 0
        black_count = 0
        queue = deque([start])
        colors[start] = 1  # Start with white
        white_count += 1
        component_nodes = [start]
        
        while queue:
            node = queue.popleft()
            for neighbor in graph[node]:
                if colors[neighbor] == 0:  # Uncolored
                    colors[neighbor] = -colors[node]  # Opposite color
                    if colors[neighbor] == 1:
                        white_count += 1
                    else:
                        black_count += 1
                    queue.append(neighbor)
                    component_nodes.append(neighbor)
                elif colors[neighbor] == colors[node]:  # Same color conflict
                    return False, []
        
        components.append((white_count, black_count, component_nodes))
    
    return True, components

def find_min_diff_and_edges(n, components):
    # Minimize |w - b| by choosing which set is white or black per component
    total_nodes = sum(w + b for w, b, _ in components)
    target = total_nodes // 2  # Ideal number of white nodes for even n
    
    # For each component, we can choose (w, b) or (b, w)
    differences = []
    for w, b, _ in components:
        # If we choose w as white, contribution to w - b is w - b
        # If we choose b as white, contribution is b - w
        differences.append((w - b, b - w))
    
    # Try to make total w - b close to 0
    min_diff = float('inf')
    for choice in range(1 << len(components)):
        total_w_minus_b = 0
        for i, (w_minus_b, b_minus_w) in enumerate(differences):
            if choice & (1 << i):
                total_w_minus_b += w_minus_b  # w as white
            else:
                total_w_minus_b += b_minus_w  # b as white
        min_diff = min(min_diff, abs(total_w_minus_b))
    
    # Number of edges needed: c - 1
    c = len(components)
    k = c - 1
    new_edges = []
    
    if k > 0:
        # Connect components by adding edges between nodes of opposite colors
        for i in range(1, c):
            node1 = components[i-1][2][0]  # First node of component i-1
            node2 = components[i][2][0]   # First node of component i
            new_edges.append((node1, node2))
    
    return min_diff, k, new_edges

def main():
    # Read input
    n, m = map(int, input().split())
    graph = defaultdict(list)
    for _ in range(m):
        u, v = map(int, input().split())
        graph[u].append(v)
        graph[v].append(u)
    
    # Check if graph is bipartite and get components
    is_bipartite, components = is_bipartite_and_count(graph, n)
    
    if not is_bipartite:
        print("Impossible")  # Should not happen per constraints
        return
    
    # Find minimum |w - b| and edges to add
    min_diff, k, new_edges = find_min_diff_and_edges(n, components)
    
    # Output
    print(f"{min_diff} {k}")
    for u, v in new_edges:
        print(f"{u} {v}")

if __name__ == "__main__":
    main()

ValueError: not enough values to unpack (expected 2, got 1)

In [2]:
from collections import defaultdict, deque
from math import ceil, floor

def find_components_and_colors(n, graph):
    colors = [0] * (n + 1)  # 0: uncolored, 1: white, -1: black
    components = []
    
    for start in range(1, n + 1):
        if colors[start] != 0:
            continue
        # BFS to color component and count nodes
        white_count = 0
        black_count = 0
        queue = deque([start])
        colors[start] = 1
        white_count += 1
        component_nodes = {start}
        
        while queue:
            node = queue.popleft()
            for neighbor in graph[node]:
                if colors[neighbor] == 0:
                    colors[neighbor] = -colors[node]
                    if colors[neighbor] == 1:
                        white_count += 1
                    else:
                        black_count += 1
                    queue.append(neighbor)
                    component_nodes.add(neighbor)
                elif colors[neighbor] == colors[node]:
                    return False, []
        
        components.append((white_count, black_count, component_nodes))
    
    return True, components

def minimize_diff(n, components):
    # Target: w ≈ b ≈ n/2
    target = n // 2
    white_sums = []
    
    # For each component, we can choose white_count or black_count as white
    for w, b, _ in components:
        white_sums.append((w, b))
    
    # Try all combinations to minimize |w - b|
    min_diff = float('inf')
    best_white = 0
    for choice in range(1 << len(components)):
        total_white = 0
        for i, (w, b) in enumerate(white_sums):
            if choice & (1 << i):
                total_white += w
            else:
                total_white += b
        total_black = n - total_white
        diff = abs(total_white - total_black)
        if diff < min_diff:
            min_diff = diff
            best_white = total_white
    
    return min_diff, best_white

def main():
    # Read input
    n, m = map(int, input().split())
    graph = defaultdict(list)
    for _ in range(m):
        u, v = map(int, input().split())
        graph[u].append(v)
        graph[v].append(u)
    
    # Find components and check bipartiteness
    is_bipartite, components = find_components_and_colors(n, graph)
    if not is_bipartite:
        print("Impossible")
        return
    
    # Minimize |w - b|
    min_diff, total_white = minimize_diff(n, components)
    
    # Connect components
    k = len(components) - 1
    new_edges = []
    if k > 0:
        # Recompute colors to ensure new edges connect opposite colors
        colors = [0] * (n + 1)
        white_count = 0
        for i, (w, b, nodes) in enumerate(components):
            start_node = next(iter(nodes))
            queue = deque([start_node])
            # Choose coloring to align with total_white
            if white_count < total_white and w <= total_white - white_count:
                colors[start_node] = 1
                white_count += w
            else:
                colors[start_node] = -1
                white_count += b
            
            while queue:
                node = queue.popleft()
                for neighbor in graph[node]:
                    if colors[neighbor] == 0:
                        colors[neighbor] = -colors[node]
                        queue.append(neighbor)
        
        # Add k edges
        for i in range(1, len(components)):
            node1 = next(iter(components[i-1][2]))
            node2 = next(iter(components[i][2]))
            # Ensure opposite colors
            if colors[node1] == colors[node2]:
                # Swap colors in component i
                for node in components[i][2]:
                    colors[node] = -colors[node]
            new_edges.append((node1, node2))
    
    # Output
    print(f"{min_diff} {k}")
    for u, v in new_edges:
        print(f"{u} {v}")

if __name__ == "__main__":
    main()

ValueError: not enough values to unpack (expected 2, got 1)